In [1]:
# process_utils.py
import psutil
import os
import platform


def find_process_by_port(port):
    for proc in psutil.process_iter(["pid", "name"]):
        try:
            connections = proc.connections(kind="inet")
            for conn in connections:
                if conn.laddr.port == port:
                    return proc.pid
        except (psutil.AccessDenied, psutil.NoSuchProcess):
            continue
    return None


def kill_process(pid):
    try:
        if platform.system() == "Windows":
            os.system(f"taskkill /PID {pid} /F")
        else:
            os.system(f"kill -9 {pid}")
        print(f"Process with PID {pid} has been terminated.")
    except Exception as e:
        print(f"Error terminating process with PID {pid}: {e}")


def kill_port_process(port: int):
    pid = find_process_by_port(port)

    if pid:
        print(f"Process found using port {port}: PID {pid}")
        kill_process(pid)
    else:
        print(f"No process found using port {port}")


pyu_port = 16307
spu_port = 11666

kill_port_process(port=pyu_port)
kill_port_process(port=spu_port)

import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

pyu_port = 16307
spu_port = 11666


cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "alice",
}

tls_config = {
    "ca_cert": "/home/beng003/certificate/bob_ca.crt",
    "cert": "/home/beng003/certificate/alice_server_cert.crt",
    "key": "/home/beng003/certificate/alice_server_key.key",
}


sf.init(address="ecm-01:6379", cluster_config=cluster_config, tls_config=tls_config)
alice = sf.PYU("alice")
bob = sf.PYU("bob")
print("Alice and Bob are ready to go!")

/tmp/ipykernel_20529/791502572.py:10: DeprecationWarning: connections() is deprecated and will be removed; use net_connections() instead
  connections = proc.connections(kind="inet")


No process found using port 16307
No process found using port 11666


2024-08-13 19:23:26,550	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: ecm-01:6379...
2024-08-13 19:23:26,565	INFO worker.py:1724 -- Connected to Ray cluster.
2024-08-13 19:23:26.602 INFO api.py:233 [alice] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': '0.0.0.0:16307', 'bob': 'ecm-02:16307'}, 'CURRENT_PARTY_NAME': 'alice', 'TLS_CONFIG': {'ca_cert': '/home/beng003/certificate/bob_ca.crt', 'cert': '/home/beng003/certificate/alice_server_cert.crt', 'key': '/home/beng003/certificate/alice_server_key.key'}}


The version of SecretFlow: 1.8.0b0


2024-08-13 19:23:27.631 INFO barriers.py:284 [alice] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=20618) 2024-08-13 19:23:27.627 INFO grpc_proxy.py:359 [alice] -- [Anonymous_job] ReceiverProxy binding port 16307, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["UNAVAILABLE"]}}]}'))...
(ReceiverProxyActor pid=20618) 2024-08-13 19:23:27.630 INFO grpc_proxy.py:379 [alice] -- [Anonymous_job] Successfully start Grpc service with credentials.
2024-08-13 19:23:28.612 INFO barriers.py:333 [alice] -- [Anonymous_job] SenderProxyActor has successfully created.
2024-08-13 19:23:28.613 INFO barriers.py:520 [alice] -- [Anonymous_job] Try ping ['bo

Alice and Bob are ready to go!


## Data Preparation
We use [brease canser](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)) as our dataset.

Let us build a mix partitioned data with this dataset. The partitions are as follows:

|label|feature1\~feature2|feature3\~feature4|
|---|---|---|
|alice_y0|alice_x0|bob_x0|

Alice holds all label and all 1~10 features, bob holds all 11~20 fetures


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

features, label = load_breast_cancer(return_X_y=True, as_frame=True)
features.iloc[:, :] = StandardScaler().fit_transform(features)
label = label.to_frame()
label=label.iloc[0:5]
label["target"][0]=1

feat_list = [
    features.iloc[:, :2],
    features.iloc[:, 2:4],
]

alice_y0 = label.iloc[0:5]
alice_x0 = feat_list[0].iloc[0:5, :]
bob_x0 = feat_list[1].iloc[0:5, :]

In [3]:
import tempfile

tmp_dir = "/home/beng003/python_project/sf-test/data"


def filepath(filename):
    return f"{tmp_dir}/{filename}"+".csv"


alice_y0_file = filepath("alice_y0")
alice_x0_file = filepath("alice_x0")
bob_x0_file = filepath("bob_x0")

alice_x0.to_csv(alice_x0_file, index=False)
bob_x0.to_csv(bob_x0_file, index=False)
alice_y0.to_csv(alice_y0_file, index=False)

In [4]:
vdf_x0 = sf.data.vertical.read_csv(
    {alice: alice_x0_file, bob: bob_x0_file}
)
vdf_y0 = sf.data.vertical.read_csv({alice: alice_y0_file})


2024-08-13 19:23:49.003 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
2024-08-13 19:23:49.136 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.
2024-08-13 19:23:51.255 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.


In [5]:
from typing import List

from secretflow.security.aggregation import SecureAggregator
import spu


def heu_config(sk_keeper: str, evaluators: List[str]):
    return {
        'sk_keeper': {'party': sk_keeper},
        'evaluators': [{'party': evaluator} for evaluator in evaluators],
        'mode': 'PHEU',
        'he_parameters': {
            'schema': 'paillier',
            'key_pair': {'generate': {'bit_size': 2048}},
        },
    }


heu0 = sf.HEU(heu_config('alice', ['bob', 'alice']), spu.spu_pb2.FM128)
aggregator0 = SecureAggregator(alice, [alice, bob])


2024-08-13 19:24:03.664 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party alice.
2024-08-13 19:24:03.683 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party bob.


In [6]:
import logging

logging.root.setLevel(level=logging.INFO)

from secretflow.ml.linear import FlLogisticRegressionVertical

model = FlLogisticRegressionVertical(
    devices=[alice, bob],
    aggregator=aggregator0,
    heu=heu0,
)

model.fit(
    vdf_x0,
    vdf_y0,
    batch_size=5,
    epochs=3,
    learning_rate=0.1,
)

2024-08-13 19:24:11.000 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPYUFlLrVWorker'> with party alice.
2024-08-13 19:24:11.041 INFO proxy.py:187 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPYUFlLrVWorker'> with party bob.
2024-08-13 19:24:12.854 INFO fl_lr_v.py:437 [alice] -- [Anonymous_job] Epoch 0: loss = [[0.83582791]]
2024-08-13 19:24:12.956 INFO fl_lr_v.py:437 [alice] -- [Anonymous_job] Epoch 1: loss = [[0.76578909]]
2024-08-13 19:24:13.037 INFO fl_lr_v.py:437 [alice] -- [Anonymous_job] Epoch 2: loss = [[0.71598927]]
2024-08-13 19:24:13.120 INFO fl_lr_v.py:443 [alice] -- [Anonymous_job] Epoch 3: loss = [[0.67995107]]


(SenderProxyActor pid=20673) [2024-08-13 19:24:12.894] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7


In [7]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_pred = np.concatenate(sf.reveal(model.predict(vdf_x0)))

auc = roc_auc_score(label.values, y_pred)
acc = np.mean((y_pred > 0.5) == label.values)
print('auc:', auc, ', acc:', acc)

auc: 0.75 , acc: 0.68
